In [2]:
# Loaded variable 'df' from URI: c:\Users\hp\IA\data2.csv
import pandas as pd
df = pd.read_csv("data2.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            1319 non-null   int64  
 1   gender         1319 non-null   int64  
 2   status         1319 non-null   object 
 3   pressurehight  1319 non-null   int64  
 4   pressurelow    1319 non-null   int64  
 5   glucose        1319 non-null   float64
 6   kcm            1319 non-null   float64
 7   troponin       1319 non-null   float64
 8   impluse        1319 non-null   int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 92.9+ KB


In [3]:
df.isnull().sum()

age              0
gender           0
status           0
pressurehight    0
pressurelow      0
glucose          0
kcm              0
troponin         0
impluse          0
dtype: int64

In [4]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1314    False
1315    False
1316    False
1317    False
1318    False
Length: 1319, dtype: bool

In [5]:
from sklearn.model_selection import train_test_split
X=df.drop(columns=['status'])
y=df['status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()


In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
import joblib

# Features et target
X = df[num_cols]  # tes colonnes numériques
y = df['status']  # remplace par ta colonne à prédire

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Encoder la cible
y_train = y_train.map({'negative': 0, 'positive': 1})
y_test = y_test.map({'negative': 0, 'positive': 1})
# Préprocesseur
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols)
])

# Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state=42))
])

# GridSearch
param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=1
)

# Train
grid_search.fit(X_train, y_train)

# Résultats
print("Best Parameters:", grid_search.best_params_)

y_pred = grid_search.best_estimator_.predict(X_test)
# Évaluation
print("Accuracy :", accuracy_score(y_test, y_pred))
print(" F1-score :", f1_score(y_test, y_pred))

# Sauvegarder le meilleur modèle utilisant joblib
joblib.dump(grid_search.best_estimator_, "model_risk.pkl")
print(" Modèle sauvegardé sous model_risk.pkl")

Best Parameters: {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__n_estimators': 100}
Accuracy : 0.9810606060606061
 F1-score : 0.9847094801223242
 Modèle sauvegardé sous model_risk.pkl
